In [15]:
pip show pyautogen

Name: pyautogen
Version: 0.8.4
Summary: A programming framework for agentic AI
Home-page: 
Author: 
Author-email: Chi Wang & Qingyun Wu <support@ag2.ai>
License: 
Location: c:\users\memo_\appdata\local\programs\python\python310\lib\site-packages
Requires: anyio, asyncer, diskcache, docker, httpx, packaging, pydantic, python-dotenv, termcolor, tiktoken
Required-by: autogen
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install --upgrade pyautogen

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install -U "autogen-agentchat"



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!pip install "autogen-ext[openai]"



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import json
from autogen import AssistantAgent, GroupChat, GroupChatManager, UserProxyAgent, config_list_from_json, register_function, ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor
import re
import datetime # Zaman damgası için eklendi
from duckduckgo_search import DDGS

# --- Yer Tutucu Fonksiyonlar ---
# Bu fonksiyonlar gerçek işlemleri simüle eder. Gerçek uygulamada API çağrıları vb. içerirler.

def log_event(event_details: str) -> str:
    """
    Belirtilen olayı zaman damgasıyla birlikte bir log dosyasına yazar.
    Args:
        event_details: Loglanacak metin mesajı.
    Returns:
        Loglama işleminin başarılı olduğunu belirten bir mesaj.
    """
    log_file = "assistant_log.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(f"[{timestamp}] {event_details}\n")
        return f"Event logged successfully to {log_file}: {event_details}"
    except Exception as e:
        print(f"Error logging event: {e}")
        return f"Error logging event: {e}"

# Web Arama Fonksiyonu (DuckDuckGo kullanarak)
def perform_web_search(query: str, max_results: int = 3) -> str:
    """
    DuckDuckGo kullanarak web araması yapar ve sonuçları döndürür.
    Args:
        query: Aranacak sorgu.
        max_results: Döndürülecek maksimum sonuç sayısı.
    Returns:
        Bulunan arama sonuçlarını içeren bir metin veya hata mesajı.
    """
    print(f"[Web Arama] DuckDuckGo ile aranıyor: '{query}' (Maks {max_results} sonuç)")
    try:
        with DDGS() as ddgs:
            # ddgs.text metodu bir jeneratördür, listeye çevirip ilk sonuçları alıyoruz
            results = list(ddgs.text(query, max_results=max_results))
            if not results:
                return f"No search results found for '{query}'."
            # Sonuçları daha okunabilir bir formatta birleştirelim
            formatted_results = "\n\n".join([f"Title: {res.get('title', 'N/A')}\nLink: {res.get('href', 'N/A')}\nSnippet: {res.get('body', 'N/A')}" for res in results])
            return f"Search results for '{query}':\n{formatted_results}"
    except Exception as e:
        print(f"[Hata] Web araması sırasında hata oluştu: {e}")
        return f"Error during web search for '{query}': {e}"

# E-posta Taslağını Dosyaya Kaydetme Fonksiyonu
def save_email_draft(to: str, subject: str, body: str) -> str:
    """
    Oluşturulan e-posta taslağını bir metin dosyasına kaydeder.
    Args:
        to: Alıcı e-posta adresi.
        subject: E-posta konusu.
        body: E-posta içeriği (LLM tarafından oluşturulmuş).
    Returns:
        İşlemin başarılı olduğunu ve dosya adını belirten bir mesaj.
    """
    draft_file = "email_drafts.txt"
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    draft_content = f"""--- Draft Saved: {timestamp} ---
To: {to}
Subject: {subject}
--- Body ---
{body}
--- End Draft ---
"""
    print(f"[E-posta Taslağı] Taslak '{draft_file}' dosyasına kaydediliyor...")
    try:
        with open(draft_file, "a", encoding="utf-8") as f:
            f.write(draft_content + "\n\n")
        return f"Email draft for {to} with subject '{subject}' saved to {draft_file}."
    except Exception as e:
        print(f"[Hata] E-posta taslağı kaydedilirken hata oluştu: {e}")
        return f"Error saving email draft: {e}"

# Yapılacaklar Listesine Öğe Ekleme Fonksiyonu (Dosyaya Yazma)
def add_todo_item(task: str) -> str:
    """
    Yapılacaklar listesine (todo_list.txt) yeni bir görev ekler.
    Args:
        task: Eklenecek görev açıklaması.
    Returns:
        İşlemin başarılı olduğunu belirten bir mesaj.
    """
    todo_file = "todo_list.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    print(f"[Yapılacaklar Listesi] Görev '{todo_file}' dosyasına ekleniyor: '{task}'")
    try:
        with open(todo_file, "a", encoding="utf-8") as f:
            f.write(f"[{timestamp}] - {task}\n")
        return f"Task '{task}' added to the To-Do list ({todo_file})."
    except Exception as e:
        print(f"[Hata] Yapılacaklar listesine eklenirken hata oluştu: {e}")
        return f"Error adding task to To-Do list: {e}"

# --- Yapılandırma ---

# 1. LLM Yapılandırması (OpenAI API Anahtarı ile)
# OAI_CONFIG_LIST.json dosyasının doğru yolda olduğundan emin olun!
config_list_path = "OAI_CONFIG_LIST.json" # JSON dosyanızın tam yolunu buraya yazın
try:
    config_list = config_list_from_json(env_or_file=config_list_path)
except FileNotFoundError:
    print(f"Hata: Yapılandırma dosyası bulunamadı: {config_list_path}")
    print("Lütfen OAI_CONFIG_LIST.json dosyasını oluşturun ve doğru API anahtarını ekleyin.")
    exit()
except json.JSONDecodeError:
    print(f"Hata: Yapılandırma dosyası ({config_list_path}) geçerli bir JSON değil.")
    exit()
except ValueError as e:
    print(f"Hata: Yapılandırma dosyasında eksik veya yanlış anahtar: {e}")
    exit()


llm_config = {
    "config_list": config_list,
    "temperature": 0.5,
    "cache_seed": 42, # Tekrarlanabilirlik için önbellekleme
    # "timeout": 600, # Gerekirse zaman aşımı süresini ayarlayın
}

# --- Ajan Tanımları ---

# 2. Ajanlar (Roller İngilizce, Yorumlar Türkçe)

# Orkestratör / Yönetici Ajan - Merkezi koordinatör
orchestrator = AssistantAgent(
    name="Orchestrator",
    llm_config=llm_config,
    system_message="""You are the Orchestrator/Supervisor Agent.
Your primary role is to manage the workflow of a multi-agent personal assistant system.
1. Receive the initial user query from the User_Proxy.
2. Delegate to 'Input_Preprocessor'.
3. Delegate to 'Intent_Task_Router' to get the plan.
4. Execute the plan step-by-step:
    - For web search: Instruct 'Web_Search_Agent' to call 'perform_web_search'.
    - For summarization: Instruct 'Summarization_Agent' with the text.
    - For scheduling: Instruct 'Scheduling_Calendar_Agent' (currently simulation).
    - For email: Instruct 'Email_Composition_Sentiment_Agent' to generate content, then instruct it to call 'save_email_draft' with the content.
    - For To-Do: Instruct 'To_Do_List_Agent' to call 'add_todo_item'.
    - For logging: Instruct 'Analytics_Logging_Agent' to call 'log_event'.
    - For clarification: Instruct 'Follow_Up_Clarification_Agent'.
5. Synthesize results for the final output.
6. When directing an agent to call a function, clearly state the function name and provide necessary arguments (e.g., "Web_Search_Agent, call perform_web_search with query='...'").
7. Prefix the final user-facing response with "FINAL_OUTPUT:".
8. End with "TERMINATE" when the entire task is complete.
Do not call functions yourself (except potentially logging). Delegate to the appropriate agent.
Available specialized agents: Input_Preprocessor, Intent_Task_Router, Web_Search_Agent, Summarization_Agent, Scheduling_Calendar_Agent, Email_Composition_Sentiment_Agent, To_Do_List_Agent, Follow_Up_Clarification_Agent, Analytics_Logging_Agent.""",
)

# --- Kullanıcı Proxy Ajanı ---
user_proxy = UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER", # Tam otomatik mod. Etkileşimli kullanım için "ALWAYS" veya "TERMINATE".
    max_consecutive_auto_reply=15, # Daha karmaşık akışlar için artırıldı
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
         "executor": LocalCommandLineCodeExecutor(work_dir="coding")
    },
    llm_config=False,
    default_auto_reply="Proceed.", # Ajanlar arasında devam etmek için varsayılan yanıt
)

# Girdi Ön İşleme Ajanı
input_preprocessor = AssistantAgent(
    name="Input_Preprocessor",
    llm_config=llm_config,
    system_message="""You are the Input Preprocessing Agent.
1. Receive the raw user query from the Orchestrator.
2. Analyze it for grammar, spelling, and meaning.
3. Identify key keywords and entities.
4. Produce a cleaned and structured version of the query.
5. Report the cleaned query back to the Orchestrator. Do not add conversational filler.
Example Response: 'Cleaned Query: [the cleaned query text]. Reporting back to Orchestrator.'""",
)

# Niyet ve Görev Yönlendirme Ajanı
intent_task_router = AssistantAgent(
    name="Intent_Task_Router",
    llm_config=llm_config,
    system_message="""You are the Intent & Task Routing Agent.
1. Receive the preprocessed query from the Orchestrator.
2. Determine the user's primary intent(s).
3. Identify which specialized task agent(s) and their required sequence are needed (Web_Search_Agent, Summarization_Agent, Scheduling_Calendar_Agent, Email_Composition_Sentiment_Agent, To_Do_List_Agent).
4. Identify the necessary inputs for each agent based on the query.
5. Report the identified intent(s), the sequence of agents/tasks, and their inputs back to the Orchestrator.
Example Response: 'Intent: Information Request, Email Composition, Task Management. Plan: 1. Web_Search_Agent (query='benefits of Autogen framework'), 2. Summarization_Agent (input=search results, format='3 bullets'), 3. Email_Composition_Sentiment_Agent (to='alex@example.com', subject='Autogen Benefits', body_context=summary), 4. To_Do_List_Agent (task='Prepare presentation on Autogen'). Reporting back to Orchestrator.'""",
)

# --- Uzmanlaşmış Görev Ajanları (Fonksiyon Haritaları ile) ---

# Web Arama Ajanı
web_search_agent = AssistantAgent(
    name="Web_Search_Agent",
    llm_config=llm_config,
    system_message="""You are the Web Search Agent.
1. Receive a specific search task and query from the Orchestrator.
2. Call the 'perform_web_search' function with the provided query.
3. Report the exact results returned by the function back to the Orchestrator.
Example Interaction: Orchestrator: "Web_Search_Agent, call perform_web_search with query='benefits of autogen'". You generate the function call and report its return value back.""",
   tools=[perform_web_search],
   reflect_on_tool_use=True
)


# Özetleme Ajanı
summarization_agent = AssistantAgent(
    name="Summarization_Agent",
    llm_config=llm_config,
    system_message="""You are the Summarization Agent.
1. Receive text (e.g., web search results) and summarization instructions (e.g., length, format) from the Orchestrator.
2. Summarize the provided text according to the instructions. Do NOT call any functions.
3. Provide the summary back to the Orchestrator.
Example Response: 'Summary (Format: 3 bullets):\n* Benefit 1\n* Benefit 2\n* Benefit 3\nReporting back to Orchestrator.'""",
)

# Zamanlama ve Takvim Ajanı (Fonksiyonsuz - Simülasyonlu)
scheduling_calendar_agent = AssistantAgent(
    name="Scheduling_Calendar_Agent",
    llm_config=llm_config,
    system_message="""You are the Scheduling & Calendar Agent.
1. Receive scheduling tasks (e.g., add event, find available time) from the Orchestrator.
2. Simulate performing the calendar operation. (No function calls defined for this agent yet).
3. Report the simulated outcome (e.g., 'Simulated: Event scheduled', 'Simulated: Conflicts found') back to the Orchestrator.
Example Response: 'Action: Simulated scheduling event '[Event Details]'. Status: Success (Simulation). Reporting back to Orchestrator.'""",
    # function_map = { "add_event": add_calendar_event_function } # Gerçek fonksiyon eklenebilir
)

# E-posta Oluşturma ve Duygu Analizi Ajanı
email_sentiment_agent = AssistantAgent(
    name="Email_Composition_Sentiment_Agent",
    llm_config=llm_config,
    system_message="""You are the Email Composition & Sentiment Analysis Agent.
1. Receive requests from the Orchestrator.
2. If asked to compose/draft an email:
    a) Generate the email body content based on the provided context.
    b) Then, call the 'save_email_draft' function, providing the recipient ('to'), subject, and the generated body content.
    c) Report the result returned by the 'save_email_draft' function back to the Orchestrator.
3. If asked to analyze sentiment: Analyze the tone (positive, negative, neutral) of the text and report the analysis back (no function call needed).
Example (Drafting): Orchestrator provides context. You generate the body. Then Orchestrator says "call save_email_draft with to='...', subject='...', body='<generated_body>'". You make the call and report the file save confirmation.""",
    tools=[save_email_draft],
    reflect_on_tool_use=True
)

# Yapılacaklar Listesi Ajanı
todo_list_agent = AssistantAgent(
    name="To_Do_List_Agent",
    llm_config=llm_config,
    system_message="""You are the To-Do List Agent.
1. Receive a task description from the Orchestrator to add to the to-do list.
2. Call the 'add_todo_item' function with the exact task description provided.
3. Report the result returned by the 'add_todo_item' function back to the Orchestrator.
Example: Orchestrator: "To_Do_List_Agent, call add_todo_item with task='Prepare presentation'". You make the call and report the confirmation message.""",
    tools=[add_todo_item],
    reflect_on_tool_use=True
)

# --- Destekleyici Ajanlar ---

# Takip ve Açıklama Ajanı
# Çalışma Mantığı: Orkestratör, bir görev için bilginin eksik olduğunu (ya bir ajan tarafından belirtildiğinde ya da kendisi fark ettiğinde)
# anlarsa, bu ajana başvurur. Bu ajan, kullanıcıya sorulacak soruyu formüle eder ve Orkestratör'e geri bildirir.
# Orkestratör soruyu User_Proxy aracılığıyla kullanıcıya iletir ve yanıtı aldığında ilgili ajana geri döner.
# Bu nedenle, bu ajanın diğer tüm ajanlara doğrudan bağlanması gerekmez; Orkestratör merkezidir.
follow_up_clarification_agent = AssistantAgent(
    name="Follow_Up_Clarification_Agent",
    llm_config=llm_config,
    system_message="""You are the Follow-Up & Clarification Agent.
1. Receive instructions from the Orchestrator when more information or user confirmation is needed to proceed with a task.
2. Formulate a clear and concise question for the user based on the Orchestrator's request.
3. Report *only the question* back to the Orchestrator. Do not ask the question directly or add conversational filler.
Example Response: 'What is the email address of the recipient?' Reporting back to Orchestrator.""",
)

# Analiz ve Loglama Ajanı
analytics_logging_agent = AssistantAgent(
    name="Analytics_Logging_Agent",
    llm_config=llm_config,
    system_message="""You are the Analytics & Logging Agent.
1. Receive instructions from the Orchestrator to log specific events or details.
2. Call the 'log_event' function with the exact details provided by the Orchestrator.
3. Report the success message returned by the 'log_event' function back to the Orchestrator.""",
    tools=[log_event],
    reflect_on_tool_use=True
)

# --- Grup Sohbeti ve Konuşmacı Seçimi ---

# Sohbetteki ajanların listesi (User_Proxy hariç, o başlatır)
agents = [
    orchestrator,
    input_preprocessor,
    intent_task_router,
    web_search_agent,
    summarization_agent,
    scheduling_calendar_agent,
    email_sentiment_agent,
    todo_list_agent,
    follow_up_clarification_agent,
    analytics_logging_agent,
]

# Orkestratör'ün yönlendirmesine dayalı özel konuşmacı seçim fonksiyonu
def select_speaker(last_speaker, groupchat):
    messages = groupchat.messages
    if len(messages) <= 1 : # İlk mesaj User_Proxy'den gelir, ikincisi Orkestratör olmalı
        return orchestrator

    last_message_content = messages[-1]["content"]
    last_speaker_name = messages[-1]["name"]

    # Orkestratör'den gelen son mesaj FINAL_OUTPUT veya TERMINATE ise sohbet biter
    if last_speaker_name == "Orchestrator" and (last_message_content.startswith("FINAL_OUTPUT:") or last_message_content.endswith("TERMINATE")):
        return None # Başka konuşmacı yok

    # Eğer konuşan Orkestratör değilse, sıra her zaman Orkestratör'e geri döner
    if last_speaker_name != "Orchestrator":
        # print(f"[Debug] Returning control to Orchestrator from {last_speaker_name}") # Hata ayıklama
        return orchestrator

    # Eğer konuşan Orkestratör ise, mesajında belirttiği bir sonraki ajanı bul
    else:
        agent_names = [agent.name for agent in agents]
        # Basit regex ile ajan isimlerini bulma (Örn: "Agent_Name, please...")
        # Bu, LLM'in yönlendirmeyi nasıl yaptığına bağlıdır.
        matches = re.findall(r'\b([A-Za-z_]+(?:Agent|Preprocessor|Router|Orchestrator))\b', last_message_content)

        if matches:
            # Mesajda bahsedilen ilk geçerli ajan ismini seç
            for name in matches:
                 for agent in agents:
                     # Orkestratörün kendisini tekrar seçmesini engelle (genellikle)
                     if agent.name == name and name != "Orchestrator":
                         # print(f"[Debug] Orchestrator selected: {agent.name}") # Hata ayıklama
                         return agent # Belirtilen ajanı seç

        # Eğer Orkestratör belirli bir ajan belirtmediyse veya format yanlışsa,
        # tekrar Orkestratör konuşsun (belki durumu düzeltir veya bitirir).
        # print(f"[Debug] No specific agent delegation found or fallback. Returning to Orchestrator.") # Hata ayıklama
        return orchestrator


# GroupChat oluşturma
groupchat = GroupChat(
    agents=agents + [user_proxy], # UserProxy'yi de ekleyebiliriz, ancak seçimi select_speaker kontrol eder
    messages=[],
    max_round=30, # Karmaşık akışlar için maksimum tur sayısı artırıldı
    speaker_selection_method=select_speaker,
    allow_repeat_speaker=[orchestrator] # Orkestratörün gerekirse art arda konuşmasına izin ver
)

# GroupChatManager oluşturma
manager = GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
    # Yönetici sistem mesajı basit olabilir, ana mantık Orkestratör ajanında
    system_message="Manage the conversation flow based on the Orchestrator's instructions.",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE")
)

# --- Sohbeti Başlatma ---

# Örnek Kullanıcı Sorgusu
user_query = (
    "Can you search the web for the main benefits of using the Autogen framework for multi-agent systems, "
    "summarize the key points into 3 bullets, and then draft an email to my colleague Alex (alex@example.com) "
    "explaining these benefits? Also, add 'Prepare presentation on Autogen' to my to-do list."
)

print(f"--- Sohbet Başlatılıyor --- \nSorgu: {user_query}\n{'='*30}")

# Kullanıcı Proxy'si aracılığıyla sohbeti başlat
# İlk mesajı doğrudan Orkestratör'e yönlendiriyoruz
initial_message = f"Orchestrator, please process the following user request: {user_query}"

user_proxy.initiate_chat(
    manager, # Ajanları koordine eden GroupChatManager
    message=initial_message,
)

# --- Son Çıktıyı İşleme (İsteğe Bağlı) ---
print(f"\n{'='*30}\n--- Sohbet Sona Erdi ---")

# Orkestratör'den gelen nihai çıktıyı ayıkla
final_output = "Nihai çıktı algılanamadı."
log_summary = []

for msg in reversed(groupchat.messages):
    # Log mesajlarını topla (isteğe bağlı)
    if msg["name"] == "Analytics_Logging_Agent" and "logged successfully" in msg["content"]:
         log_summary.append(msg["content"])

    # Nihai çıktıyı bul
    if msg["name"] == "Orchestrator" and msg["content"].startswith("FINAL_OUTPUT:"):
        final_output = msg["content"].replace("FINAL_OUTPUT:", "").strip()
        break
    # Veya TERMINATE mesajından önceki Orkestratör mesajını al
    elif msg["name"] == "Orchestrator" and msg["content"].endswith("TERMINATE") and final_output == "Nihai çıktı algılanamadı.":
         term_index = groupchat.messages.index(msg)
         if term_index > 0:
             prev_msg = groupchat.messages[term_index - 1]
             # Bazen önceki mesaj UserProxy'den gelen "Proceed." olabilir, onu atla
             if prev_msg["name"] == "Orchestrator":
                 final_output = f"(TERMINATE öncesi son Orkestratör mesajından çıkarıldı):\n{prev_msg['content']}"
             elif term_index > 1 and groupchat.messages[term_index - 2]["name"] == "Orchestrator":
                 final_output = f"(TERMINATE öncesi sondan bir önceki Orkestratör mesajından çıkarıldı):\n{groupchat.messages[term_index - 2]['content']}"


print(f"\n--- Nihai Çıktı ---")
print(final_output)

# İsteğe bağlı: Log özetini göster
# print("\n--- Loglanan Olaylar Özeti ---")
# if log_summary:
#     for log_entry in reversed(log_summary): # Kronolojik sıra için ters çevir
#         print(f"- {log_entry}")
# else:
#     print("Loglanan olay bulunamadı.")

# Log dosyasının varlığını kontrol et
if os.path.exists("assistant_log.txt"):
    print("\nLog detayları 'assistant_log.txt' dosyasına yazıldı.")

TypeError: ConversableAgent.__init__() got an unexpected keyword argument 'tools'